In [20]:
from models import unetnc
import numpy as np
import pickle as pkl
import cv2
import torch
import cv2

DATA_PATH = 'Dataset Preview/Inv3D preview/data/'
# warped WC sind die 3D Koordinaten nach dem ersten Modell bzw. dewarpnet -> Ground Truth

In [35]:
from pathlib import Path
data_dir= 'Dataset Preview/Inv3D preview complete/data/train/'
p = Path('./')
folder = list(p.glob(data_dir + '*/'))

In [36]:
folder

[PosixPath('Dataset Preview/Inv3D preview complete/data/train/06'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/07'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/09'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/01'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/05'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/04'),
 PosixPath('Dataset Preview/Inv3D preview complete/data/train/08')]

In [19]:
for item in folder:
    path = str(item) + '/warped_WC.npz')
    wc_gt = np.load(path)['warped_WC']


Dataset Preview/Inv3D preview complete/data/train/06/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/07/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/09/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/01/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/05/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/04/warped_WC.npz
Dataset Preview/Inv3D preview complete/data/train/08/warped_WC.npz


In [21]:

trainimage_gt = np.load(DATA_PATH + 'train/06/warped_WC.npz')

In [22]:
img_t = trainimage_gt['warped_WC']
#img_t = torch.from_numpy(img).float()

In [31]:

img_t = img_t.transpose(2,0,1)

In [32]:
img_t[0].shape

(256, 256)

In [34]:
np.amax(img_t[0])
np.amin(img_t[0])

-0.26865804

In [64]:
#normalize label
lbl = img_t
msk=((lbl[:,:,0]!=0)&(lbl[:,:,1]!=0)&(lbl[:,:,2]!=0)).astype(np.uint8)*255
xmx, xmn, ymx, ymn,zmx, zmn= 1.2539363, -1.2442188, 1.2396319, -1.2289206, 0.6436657, -0.67492497   # calculate from all the wcs
lbl[:,:,0]= (lbl[:,:,0]-zmn)/(zmx-zmn)
lbl[:,:,1]= (lbl[:,:,1]-ymn)/(ymx-ymn)
lbl[:,:,2]= (lbl[:,:,2]-xmn)/(xmx-xmn)
lbl=cv2.bitwise_and(lbl,lbl,mask=msk)
#lbl = cv2.resize(lbl, self.img_size, interpolation=cv2.INTER_NEAREST)
lbl = lbl.transpose(2, 0, 1)   # NHWC -> NCHW
lbl = np.array(lbl, dtype=np.float64)
lbl

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [65]:
lbl.shape

(3, 256, 256)

In [79]:
type(trainimage_gt['warped_WC'])

numpy.ndarray

In [5]:
#np.load(DATA_PATH + 'train/06/warped_document.png')

In [87]:
from numpy import asarray
from PIL import Image
image = Image.open(DATA_PATH + 'train/01/warped_document.png')
# convert image to numpy array
img = asarray(image)
#img = m.imresize(img, self.img_size) # uint8 with RGB mode
if img.shape[-1] == 4:
    img=img[:,:,:3]   # Discard the alpha channel  
img = img[:, :, ::-1] # RGB -> BGR
# plt.imshow(img)
# plt.show()
img = img.astype(float) / 255.0
img = img.transpose(2, 0, 1) # NHWC -> NCHW


In [89]:
img.shape

(3, 256, 256)

In [6]:
DATA_PATH = 'Dataset Preview/Inv3D preview/data/'

from custom_dataset import CustomImageDataset_wc
train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=True)
#train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=False)

from models import unetnc
model = unetnc.UnetGenerator(input_nc=3, output_nc=6, num_downs=3)
#model.to('cuda')
from torch.autograd import Variable

In [7]:
#for param in model.parameters():
 #   param.requires_grad = False
#for param in model.parameters():
    #print(param)

In [8]:
from torch.utils import data
#data_loader = get_loader('doc3dwc')
#data_path = args.data_path
#t_loader = data_loader(data_path, is_transform=True, img_size=(args.img_rows, args.img_cols), augmentations=True)
#v_loader = data_loader(data_path, is_transform=True, split='val', img_size=(args.img_rows, args.img_cols))

#n_classes = t_loader.n_classes
trainloader = data.DataLoader(train_dataset, batch_size=1, num_workers=8, shuffle=True)
#valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=8)

In [9]:
for image, labels in trainloader:
    #print(item)
    pass
#img = Variable(item['image'])#.to('cuda'))  #images = Variable(images.cuda())
#img.size()

In [12]:
labels

{'wc_gt': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]),
 'curvature_gt': ['TODO'],
 'text_mask_gt': ['TODO'],
 'local_angle_gt': ['TODO']}

In [10]:
output = model(image)

In [15]:
output.shape

torch.Size([1, 6, 256, 256])

In [17]:
output[0].shape

torch.Size([2, 256, 256])

In [ ]:
from torch.autograd import Variable
for i, (images, labels) in enumerate(trainloader):
    print(i)
    print(images)
    print(labels)
    #images = Variable(images.cuda())
    #labels = Variable(labels.cuda())

    #optimizer.zero_grad()
    outputs = model(images)
    '''pred=htan(outputs)
    g_loss=gloss(pred, labels)
    l1loss = loss_fn(pred, labels)
    loss=l1loss#+(0.2*g_loss)
    avg_l1loss+=float(l1loss)
    avg_gloss+=float(g_loss)
    avg_loss+=float(loss)
    train_mse+=float(MSE(pred, labels).item())

    loss.backward()
    optimizer.step()
    global_step+=1'''
    print('lakshooOOOO')
    print(outputs)

In [ ]:
model = MyLightningModule()

trainer = Trainer()
trainer.fit(model, train_dataloader, val_dataloader)
trainer.validate(val_dataloaders=val_dataloaders)
trainer.test(test_dataloaders=test_dataloaders)

### Pytorch Lightning Training

In [2]:
import pytorch_lightning as pl
DATA_PATH = 'Dataset Preview/Inv3D preview complete/data/'
data_dir= 'Dataset Preview/Inv3D preview complete/data/train/'

from custom_dataset import CustomImageDataset_wc
dataset_train = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=True)
dataset_val = CustomImageDataset_wc(data_dir=DATA_PATH+'val/', transform=True)
dataset_test = CustomImageDataset_wc(data_dir=DATA_PATH+'test/', transform=True)

from torch.utils.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size= 2, num_workers=12)
val_loader = DataLoader(dataset_val, batch_size= 1, num_workers=12)
test_loader = DataLoader(dataset_test, batch_size= 1, num_workers=12)

from models import unetnc
model = unetnc.Estimator3d(input_nc=3, output_nc=8, num_downs=3)



# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(gpus=1, max_epochs = 100)
#trainer = pl.Trainer(auto_select_gpus = True, max_epochs = 100)
#trainer.fit(model, train_loader)

# Wie sieht der Batch aus, der vom trainer und train_loader generiert wird?

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [5]:
import torch
torch.cuda.get_device_name()

'GeForce RTX 3070'

In [3]:
trainer.fit(model, train_loader)#, val_loader)

/home/leon/anaconda3/envs/rapids-0.19/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                    | Params
--------------------------------------------------
0 | model | UnetSkipConnectionBlock | 16.7 M
--------------------------------------------------
16.7 M    Trainable params
0         Non-trainable params
16.7 M    Total params
66.677    Total estimated model params size (MB)
Epoch 99: 100%|██████████| 4/4 [00:00<00:00,  5.20it/s, loss=0.116, v_num=7]


In [2]:
CustomImageDataset_wc.calculate_min_and_max(data_dir)

(1.0858383, -1.0862498, 0.8847823, -0.8838696, 0.31327668, -0.30930856)

In [4]:
for batch in train_loader:
    images, labels = batch

outputs = model(images)

In [12]:
msk=((lbl[:,:,0]!=0)&(lbl[:,:,1]!=0)&(lbl[:,:,2]!=0)).astype(np.uint8)*255

In [9]:
loss = (outputs[:,0:3,:,:] - labels['wc_gt'])

In [12]:
import torch
torch.sum(loss)

tensor(45.6411, grad_fn=<SumBackward0>)

In [8]:
labels['wc_gt']

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [15]:
lbl.shape

(256, 256, 3)

In [17]:
lbl[:,:,0].shape

(256, 256)

In [11]:
import numpy as np
lbl = np.load(data_dir + '01/warped_WC.npz')['warped_WC']

In [10]:
import torch
import math
phi_xx = outputs[:,3:4,:,:]
phi_xy = outputs[:,4:5,:,:]
phi_yx = outputs[:,5:6,:,:]
phi_yy = outputs[:,6:7,:,:]
curvature_mesh = outputs[:,7:8,:,:]

#l1_loss = torch.norm((wc_coordinates - labels['wc_gt']),p=1,dim=(1,2,3))

theta_x = torch.atan2(phi_xx, phi_xy)
theta_y = torch.atan2(phi_yx, phi_yy)
#p_x = torch.norm((phi_xx, phi_xy),p=2,dim=(1,2,3))
#p_y = torch.norm(phi_yx, phi_yy,p=2,dim=(1,2,3))
theta_x_gt = labels['warped_angle_gt'][:,0:1,:,:]
theta_y_gt = labels['warped_angle_gt'][:,1:2,:,:]

In [30]:
theta_x + theta_x_gt

tensor([[[[ 0.1338,  1.5896,  0.2824,  ...,  1.5574,  1.9771,  1.3935],
          [-1.4128,  2.0649, -1.7539,  ..., -2.2534,  1.4444,  0.5271],
          [ 2.2552,  2.6485, -0.2000,  ..., -0.6726,  1.1247, -2.8811],
          ...,
          [-1.0759,  0.2942, -0.7422,  ..., -2.2483, -0.7272, -1.2089],
          [ 1.1033, -2.7409,  1.2070,  ..., -2.8661, -0.5641,  2.1466],
          [-0.1207,  2.8677, -0.0220,  ...,  1.0390,  0.0371,  0.4418]]],


        [[[ 0.3666,  2.0270,  0.1420,  ...,  1.8535, -0.2324,  1.4483],
          [-1.1879,  2.9411, -1.5738,  ..., -2.9330,  3.0907,  0.7603],
          [ 2.0954,  2.6800, -0.4812,  ...,  1.2889, -0.3040, -1.4327],
          ...,
          [-1.7048,  1.3386, -1.0337,  ...,  1.2414, -1.6064, -1.4240],
          [ 1.1488,  3.0711, -0.0193,  ...,  1.4601, -0.1932,  2.1069],
          [ 0.3182,  1.9751,  1.7474,  ...,  0.6415,  0.4315,  0.9014]]],


        [[[ 0.0592,  2.6212,  0.1191,  ...,  1.7347, -0.3547,  1.3688],
          [-1.6484,  2.448

In [35]:
ass = torch.sub(theta_x, theta_x_gt)
l_angle = ass * labels['warped_text_mask']
l_angle.shape

torch.Size([3, 1, 256, 256])

In [4]:
 labels['warped_text_mask']

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [44]:
l_angle[:,0,1,1]

tensor([0., 0., 0.], grad_fn=<SelectBackward>)

In [42]:
wc_coord = output[:,0:3,:,:]
a = wc_coord - labels['wc_gt']
l1_loss = torch.norm(a,p=1,dim=(1,2,3))

In [47]:
torch.mean(l1_loss + l_angle[:,0,1,1])

tensor(87412.8125, grad_fn=<MeanBackward0>)

In [48]:
l_angle[:,0,:,:]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [-0., 0., -0.,  ..., -0., 0., 0.],
         [0., 0., -0.,  ..., -0., 0., -0.],
         ...,
         [-0., 0., -0.,  ..., -0., -0., -0.],
         [0., -0., 0.,  ..., -0., -0., 0.],
         [-0., 0., -0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., -0., 0.],
         [-0., 0., -0.,  ..., -0., 0., 0.],
         [0., 0., -0.,  ..., 0., -0., -0.],
         ...,
         [-0., 0., -0.,  ..., 0., -0., -0.],
         [0., 0., -0.,  ..., 0., -0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., -0., 0.],
         [-0., 0., -0.,  ..., -0., -0., 0.],
         [0., 0., -0.,  ..., 0., -0., -0.],
         ...,
         [-0., 0., -0.,  ..., 0., -0., -0.],
         [0., 0., -0.,  ..., 0., -0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<SliceBackward>)

In [49]:
l1_loss = torch.norm((wc_coord - labels['wc_gt']),p=1,dim=(1))
l1_loss.shape

torch.Size([3, 256, 256])

In [58]:
labels['warped_angle_gt']

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

In [50]:
l1_loss

tensor([[[0.5506, 0.4435, 0.4438,  ..., 0.7454, 0.7216, 0.6675],
         [0.7710, 0.8851, 0.4962,  ..., 0.3998, 1.2136, 0.8322],
         [0.8324, 1.0413, 0.3836,  ..., 1.1693, 0.9843, 1.1180],
         ...,
         [0.9629, 1.2628, 0.6848,  ..., 1.0529, 0.5139, 1.0972],
         [1.0644, 0.5942, 1.8196,  ..., 0.2718, 0.8727, 1.1694],
         [0.8080, 1.3267, 1.0267,  ..., 0.9553, 0.7356, 0.3120]],

        [[0.4196, 0.4022, 0.6593,  ..., 0.3808, 0.5979, 0.5905],
         [0.9907, 1.1944, 0.1120,  ..., 1.4419, 0.7706, 0.5274],
         [0.9510, 0.6798, 0.4665,  ..., 1.2564, 1.0754, 0.5007],
         ...,
         [0.5237, 0.7582, 0.3622,  ..., 1.1582, 0.5906, 0.8369],
         [0.2259, 0.5273, 1.3454,  ..., 0.5213, 0.4836, 0.7358],
         [0.6822, 0.8088, 1.0094,  ..., 0.1881, 0.6821, 0.2091]],

        [[0.5761, 0.3500, 0.4825,  ..., 0.4284, 0.5095, 0.5167],
         [0.7191, 0.6350, 1.0663,  ..., 1.7054, 1.0989, 0.6331],
         [0.7618, 0.6713, 0.6240,  ..., 1.1202, 1.1306, 0.

In [57]:
l_angle.squeeze().shape

torch.Size([3, 256, 256])

In [55]:
torch.mean(l1_loss)

tensor(1.3338, grad_fn=<MeanBackward0>)

In [25]:
def l_angle_def(theta_x, theta_y, theta_x_gt, theta_y_gt , type = 'paper'):
    if type == 'paper':
        l_x = (torch.abs(theta_x - theta_x_gt) - math.pi) % (2*math.pi)
        l_y = (torch.abs(theta_y - theta_y_gt) - math.pi) % (2*math.pi)
        l_angle = l_x + l_y
        return l_angle
    else:
        l_x = (torch.abs(theta_x - theta_x_gt) - math.pi) % (math.pi)
        l_y = (torch.abs(theta_y - theta_y_gt) - math.pi) % (math.pi)
        l_angle = l_x + l_y
        return l_angle
l_angle = l_angle_def(theta_x, theta_y, theta_x_gt, theta_y_gt , type = 'paper')

In [26]:
l_angle * 

tensor([[[[ 6.5502,  9.1036,  8.4098,  ..., 10.5725,  9.7732,  8.1592],
          [ 9.5878, 11.1359, 10.6318,  ..., 10.5132,  8.8400,  8.1323],
          [11.4941, 10.8336,  8.4940,  ...,  7.8338, 10.4742,  9.9351],
          ...,
          [10.0284,  7.5092,  9.4902,  ..., 10.1936,  9.6098,  9.5224],
          [ 8.5193, 10.8087,  8.1074,  ..., 10.8176,  8.2405,  8.7833],
          [ 9.2108, 11.6660,  8.8763,  ...,  9.3722,  8.8739,  7.8045]]],


        [[[ 6.9675,  9.8282,  9.1158,  ...,  9.5963,  8.0754,  8.3313],
          [ 9.7767, 11.5925, 10.3022,  ..., 11.7513, 11.6905,  9.9686],
          [ 9.8845, 10.1646,  7.2149,  ...,  8.3453,  8.1525,  7.9435],
          ...,
          [11.1207, 10.0063,  9.9313,  ...,  9.5163, 10.6405,  9.8720],
          [ 8.0595, 11.6377,  8.0761,  ..., 10.6131,  9.3898,  8.4291],
          [ 9.1431, 10.9209, 10.3663,  ...,  8.9672,  8.9980,  7.8156]]],


        [[[ 6.4337,  9.9325,  8.1331,  ...,  9.3034,  8.3387,  8.2734],
          [ 9.9492, 11.703

In [10]:
import torch.nn as nn
#import grad_loss
# Losses
#MSE = nn.MSELoss()
loss_fn = nn.L1Loss()
#gloss= grad_loss.Gradloss(window_size=5,padding=2)

# Activation
htan = nn.Hardtanh(0,1.0)

In [11]:
def split_tensors(model_output):
    batch_size = model_output.shape[0]
    wc_coordinates_pred = torch.empty(batch_size,3,256,256)
    #angle_pred = torch.empty(batch_size,4,256,256)
    phi_xx = torch.empty(batch_size,1,256,256)
    phi_xy = torch.empty(batch_size,1,256,256)
    phi_yx = torch.empty(batch_size,1,256,256)
    phi_yy = torch.empty(batch_size,1,256,256)

    idx = 0
    for output in model_output:
        
        wc_coordinates_pred[idx] = output[:3]
        #angle_pred[idx] = output[3:]
        phi_xx[idx] = output[3]
        phi_xy[idx] = output[4]
        phi_yx[idx] = output[5]
        phi_yy[idx] = output[6]

        idx += 1

    angle_pred = {'phi_xx' : phi_xx, 'phi_xy' : phi_xy, 'phi_yx' : phi_yx, 'phi_yy' : phi_yy,}
    return wc_coordinates_pred, angle_pred

In [26]:
output.shape

torch.Size([3, 7, 256, 256])

In [37]:
wc_coord = output[:,0:3,:,:]

In [12]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()
wc_coordinates_output, angle_output = split_tensors(output)

#wc_coordinates_pred=htan(wc_coordinates_output)
#l1_loss = loss_fn(wc_coordinates_pred, labels['wc_gt'])

In [14]:
l1_loss = torch.cdist(wc_coordinates_output, labels['wc_gt'], p=1)

In [39]:
a.shape

torch.Size([3, 3, 256, 256])

In [42]:
from torch import linalg as LA
a = wc_coord - labels['wc_gt']
torch.norm(a,p=1,dim=(1,2,3))
#LA.matrix_norm(a, ord=1)

#LA.norm(a, ord=1)

tensor([105090.3906,  98479.5078, 102082.2266], grad_fn=<NormBackward1>)

In [138]:

loss = l1_loss #+ torch.tensor(0.25, requires_grad=True)

In [139]:
loss.backward()
optimizer.step()
#optimizer.zero_grad()

In [141]:
l1_loss.grad

<ipython-input-141-291d24f22676>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  l1_loss.grad


In [30]:
theta_x = torch.atan2(angle_output['phi_xx'], angle_output['phi_xy'])
theta_y = torch.atan2(angle_output['phi_yx'], angle_output['phi_yy'])



In [34]:
theta_x

tensor([[[[-2.3387,  1.8802, -3.0584,  ...,  2.3485, -3.0838,  1.8430],
          [ 1.0988,  2.4272,  0.0600,  ...,  1.4874, -0.4773, -0.5839],
          [ 1.1703,  2.1403,  0.9957,  ...,  1.9287,  0.8192, -2.4067],
          ...,
          [-0.9434,  3.1323, -0.2417,  ...,  2.9774,  0.6064, -2.6378],
          [ 0.0099, -2.8343, -1.4282,  ...,  2.2990,  1.9182,  2.6005],
          [ 1.0062, -3.1223,  0.5710,  ...,  2.4470,  0.3611, -0.6768]]],


        [[[-2.5209,  1.1893, -2.5897,  ...,  2.3591,  2.8945,  1.1009],
          [ 1.3222, -3.0580, -0.1832,  ...,  1.9137, -0.6337, -2.9021],
          [ 2.5742,  2.0284,  1.1411,  ...,  2.2434,  0.8028, -2.9189],
          ...,
          [-0.7203,  2.6766, -0.1862,  ...,  2.5575,  0.6216, -2.8381],
          [ 0.3242,  2.4139,  1.7511,  ...,  1.7789,  1.3775,  2.7694],
          [ 0.9446,  2.9962,  0.5149,  ..., -2.4320,  0.1336, -1.1009]]],


        [[[-2.3946,  1.2582, -2.7747,  ...,  2.3261,  2.8953,  1.0995],
          [ 1.1989,  2.198

In [37]:
p_x = torch.cdist(angle_output['phi_xx'],angle_output['phi_xy'])
p_y = torch.cdist(angle_output['phi_yx'],angle_output['phi_yy'])

In [52]:
test = torch.cdist(angle_output['phi_xx'],angle_output['phi_xy'], p=2)

In [53]:
test

tensor([[[[ 7.2391, 10.6285, 11.2850,  ..., 13.6684, 14.6986, 10.3504],
          [ 7.2376,  8.2053,  9.3738,  ..., 11.4745, 12.9501,  9.9714],
          [11.6378, 11.5819, 12.3489,  ..., 14.1231, 16.3125, 11.9589],
          ...,
          [ 6.5999,  8.1252,  8.6379,  ..., 10.4022, 12.0047,  7.9056],
          [ 9.9543, 13.8107, 14.6640,  ..., 16.5764, 18.2547, 13.7567],
          [ 7.8080,  8.8072, 10.2071,  ..., 11.5712, 14.3958,  9.3063]]],


        [[[ 7.1878, 10.7629, 10.9981,  ..., 13.6370, 12.2194,  6.3300],
          [ 6.9235,  9.3275,  9.8757,  ..., 11.1887, 11.8747,  7.1685],
          [12.0948, 12.1321, 13.3600,  ..., 14.5807, 14.1684,  8.3100],
          ...,
          [ 7.3444, 10.6796, 11.0797,  ..., 12.8764, 13.1144,  7.4377],
          [12.4934, 12.7724, 14.0844,  ..., 15.8269, 14.5963,  8.6161],
          [ 5.9442,  8.4455,  9.1291,  ..., 11.2682, 10.7131,  4.1938]]],


        [[[ 7.6417, 12.6649, 10.1858,  ..., 14.9750, 11.8834,  6.0450],
          [ 8.5360,  9.621

In [97]:
model.parameters().grad

AttributeError: 'generator' object has no attribute 'grad'

In [144]:
torch.sum(l1_loss,torch.tensor(0.25, requires_grad=True))

TypeError: sum() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [215]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.linear = nn.Linear(2,2)
        nn.

    def forward(self, x):
        x = F.relu(self.linear(x))
        return x


net = Net()

In [148]:
print(net)

Net(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)


tensor([[1.4341, 0.0000]], grad_fn=<ReluBackward0>)


In [304]:
input = torch.randn(1, 2, requires_grad=True)
out = net(input)
print(input)
print(out)

tensor([[0.8760, 1.4145]], requires_grad=True)
tensor([[0.1553, 0.0000]], grad_fn=<ReluBackward0>)


In [305]:
loss_fn = nn.L1Loss()

emp = torch.empty(1,1)
emp[0][0] = 2
#emp[0][1] = 2'''

emp2 = torch.empty(1,2)
emp2[0][0] = 1
emp2[0][1] = 2
# Activation
htan = nn.Hardtanh(0,1.0)
t1 = torch.empty(1,1)
t1[0] = out[0][0]
t2 = torch.empty(1,1)
t2[0] = out[0][1]
#pred=htan(out)
l1_loss = loss_fn((t1+2) * 5 , emp)

In [306]:
l1_loss.backward()

In [299]:
l1_loss.grad

<ipython-input-299-291d24f22676>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  l1_loss.grad


In [307]:
input.grad

tensor([[0.2555, 2.7904]])